In [5]:
import tensorflow as tf
import cv2
import numpy as np

MODEL_PATH = "current_best.keras"
TEST_VIDEO_PATH = "test"
USE_WEBCAM = True
IMAGE_SIZE = (254, 254)
BATCH_SIZE = 8

if (USE_WEBCAM):
    video_capture_filepath = 0
else:
    video_capture_filepath = f"{TEST_VIDEO_PATH}/test2.mp4"
    
RESCALING_FACTOR = 1./0xFF

In [16]:
model = tf.keras.saving.load_model(MODEL_PATH)

2024-09-09 18:28:53.553278: E external/local_xla/xla/stream_executor/cuda/cuda_driver.cc:274] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2024-09-09 18:28:53.553311: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:129] retrieving CUDA diagnostic information for host: farbod-ASUS-TUF-Gaming-F15-FX507ZU4-FX507ZU4
2024-09-09 18:28:53.553318: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:136] hostname: farbod-ASUS-TUF-Gaming-F15-FX507ZU4-FX507ZU4
2024-09-09 18:28:53.553470: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:159] libcuda reported version is: 535.129.3
2024-09-09 18:28:53.553487: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:163] kernel reported version is: 535.129.3
2024-09-09 18:28:53.553491: I external/local_xla/xla/stream_executor/cuda/cuda_diagnostics.cc:241] kernel version seems to match DSO: 535.129.3


In [7]:
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

In [27]:
cap = cv2.VideoCapture(video_capture_filepath)

IMAGE_SIZE = (254, 254)
def isRealLabel(isReal, mean_score):
    if isReal:
        return f"Real, score: {mean_score:.2f}"
    else:
        return f"AI, score: {mean_score:.2f}"
    

def shape(ndarray) :
    if isinstance(ndarray, list):
        # More dimensions, so make a recursive call
        outermost_size = len(ndarray)
        row_shape = shape(ndarray[0])
        return (outermost_size, *row_shape)
    else:
        # No more dimensions, so we're done
        return ()
    
try:
    isReal = True
    framelist = np.zeros((10 * BATCH_SIZE, *IMAGE_SIZE, 3))
    mean_score = 0
    index = 0
    while True:
        ret, frame = cap.read()
        
        if not ret:
            break

        gray_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

        faces = face_cascade.detectMultiScale(gray_frame, scaleFactor=1.1, minNeighbors=5, minSize=(32, 32), maxSize=IMAGE_SIZE)

        x_st, y_st = 0, 0
        for (x, y, w, h) in faces:
            
            y_padding, x_padding = IMAGE_SIZE[1] - h, IMAGE_SIZE[0] - w
            y_st, x_st = int(y - y_padding // 2), int(x - x_padding // 2)
            y_end, x_end = int(y + h + (y_padding+1) // 2), int(x + w + (x_padding+1) // 2)
            
            if x_st < 0 or y_st < 0 or y_end > frame.shape[1] or x_end > frame.shape[0]:
                continue
            
            cv2.rectangle(frame, (x_st, y_st), (x_end, y_end), (255, 0, 0), 2)
            
            face_frame = np.asarray(frame[y_st: y_end, x_st: x_end] * RESCALING_FACTOR)
            framelist[index] = face_frame
            # print(f"x_st:{x_st}, y_st:{y_st}, x_end:{x_end}, y_end:{y_end}")
            index += 1

            cv2.putText(frame, isRealLabel(isReal, mean_score), (x_st, y_st), cv2.FONT_HERSHEY_DUPLEX, 1, [0, 0, 255], 2)
        cv2.imshow('Live Video Stream', frame)
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
        
        if index >= BATCH_SIZE:
            # print(shape(framelist))
            print(framelist[:index].shape)
            mean_score = np.mean(model.predict(framelist[:index], verbose=0))
            index = 0
            print(mean_score)
            isReal = mean_score > 0.5
            framelist = np.zeros((10 * BATCH_SIZE, *IMAGE_SIZE, 3))
            

        
finally:
    cap.release()
    cv2.destroyAllWindows()

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to target thread (0x55f0f0818040)

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to target thread (0x55f0f0818040)

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to target thread (0x55f0f0818040)

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to target thread (0x55f0f0818040)

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to target thread (0x55f0f0818040)

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to target thread (0x55f0f0818040)

QObject::moveToThread: Current thread (0x55f0f0818040) is not the object's thread (0x55f0eff970c0).
Cannot move to tar

(8, 254, 254, 3)
1/1 [==============================] - 1s 529ms/step
0.990909
(8, 254, 254, 3)
1/1 [==============================] - 1s 592ms/step
0.99272275
(8, 254, 254, 3)
1/1 [==============================] - 0s 457ms/step
0.9971736
(8, 254, 254, 3)
1/1 [==============================] - 0s 497ms/step
0.9972619
(9, 254, 254, 3)
1/1 [==============================] - 1s 765ms/step
0.992021
(8, 254, 254, 3)
1/1 [==============================] - 1s 551ms/step
0.99238336
(8, 254, 254, 3)
1/1 [==============================] - 1s 523ms/step
0.9901259
(8, 254, 254, 3)
1/1 [==============================] - 1s 520ms/step
0.99213505
(8, 254, 254, 3)
1/1 [==============================] - 1s 581ms/step
0.99129987
(8, 254, 254, 3)
1/1 [==============================] - 1s 517ms/step
0.99499375
(8, 254, 254, 3)
1/1 [==============================] - 1s 563ms/step
0.9970224
(8, 254, 254, 3)
1/1 [==============================] - 1s 566ms/step
0.9947133
(8, 254, 254, 3)
1/1 [===============

ValueError: could not broadcast input array from shape (212,254,3) into shape (254,254,3)